# View Plots Comparing Bispectrum Coefficients for the Systems
In this notebook, we plot histograms to compare bispectrum coefficients of the systems with 4 atoms (FeCoNiCu, CrCoNiCu, CrFeNiCu, CrFeCoCu, CrFeCoNi) and CrFeCoNiCu. The histograms allow us to compare how the bispectrum coefficients change for the different systems and better understand why the network is able to predict CrFeCoNiCu properties well when trained on specific 4-atom systems. We also plot histograms for to compare the output properties of each system.

Overview
1. Load Bispectrum Coefficients and Output Properties
2. Plot Histograms for Bispectrum Coefficients
3. Plot Histograms for Output Properties

In [ ]:
# import libraries we will need
import json as js
import numpy as np

import plotly.offline as p
import plotly.graph_objs as go

## 1. Load Bispectrum Coefficients and Output Properties
The unrelaxed bispectrum coefficients and relaxed vacancy formation energy, cohesive energy, and local atomic pressures and volumes are obtained from a JSON file and stored in a Python dictionary. The bispectrum coefficients are local geometric descriptors based on each atom's 12 nearest neighbors.

In [ ]:
# properties to get
properties = ["Relaxed_VFE", "Cohesive_Energy", "Pressure", "Volume"]
# atom systems
systems = ['FeCoNiCu', 'CrCoNiCu', 'CrFeNiCu', 'CrFeCoCu', 'CrFeCoNi', 'CrFeCoNiCu']
input_prop_key = 'Unrelaxed_Bispectrum_Coefficients'

properties_dictionaries = {}

# gather by filenames and automatically include queries for all the properties in the dictionary
for system in systems:
    filename = '../data/{}.json'.format(system)
    # open file and load data into data variable
    with open(filename, 'r') as f:
        data = js.load(f)

    # get relevant information from data variable
    elements = data['element']
    relaxed_vfe = data['Relaxed_VFE']
    cohesive_energy = data['Cohesive_Energy']
    pressure = data['Pressure']
    volume = data['Volume']
    input_properties = data[input_prop_key]

    # store input and output properties for specific element being searched for
    elements_array = np.array([]) 
    relaxed_vfe_array = np.array([])
    cohesive_energy_array = np.array([])
    pressure_array = np.array([])
    volume_array = np.array([])
    input_properties_array = np.array([])

    # create counters to track number of each element
    num_Co = 0
    num_Cr = 0
    num_Cu = 0
    num_Fe = 0
    num_Ni = 0
    
    # iterate through elements and get input and output properties for desired element
    for i, val in enumerate(elements):
        relaxed_vfe_array = np.append(relaxed_vfe_array, relaxed_vfe[i])
        cohesive_energy_array = np.append(cohesive_energy_array, cohesive_energy[i])
        pressure_array = np.append(pressure_array, pressure[i])
        volume_array = np.append(volume_array, volume[i])
        input_properties_array = np.append(input_properties_array, np.asarray(input_properties[i])) 
        if (val == 'Co'):
            elements_array = np.append(elements_array, 27)
            num_Co = num_Co + 1
        elif (val == 'Cr'):
            elements_array = np.append(elements_array, 24)
            num_Cr = num_Cr + 1
        elif (val == 'Cu'):
            elements_array = np.append(elements_array, 29)
            num_Cu = num_Cu + 1
        elif (val == 'Fe'):
            elements_array = np.append(elements_array, 26)
            num_Fe = num_Fe + 1
        elif (val == 'Ni'):
            elements_array = np.append(elements_array, 28)
            num_Ni = num_Ni + 1
 
    # reshape input_properties_element (this should only happen if input property is Bispectrum coefficients)
    num_rows = int (input_properties_array.shape[0]/55)
    input_properties_array = np.reshape(input_properties_array, (num_rows, 55))
    
    # element number is included as input to model
    elements_array = elements_array[np.newaxis].T
    input_properties_array = np.append(input_properties_array, elements_array, 1)

    input_properties_array = input_properties_array.astype(np.float)

    num_elements = np.array([num_Co, num_Cr, num_Fe, num_Ni])

    # create properties_dictionary
    properties_dictionaries[system] = {
        'inputs': input_properties_array,
        'relaxed_vfe': relaxed_vfe_array,
        'cohesive_energy': cohesive_energy_array,
        'pressure': pressure_array,
        'volume': volume_array,
        'length': relaxed_vfe_array.shape[0],
        'elements': elements_array,
    }

## 2. Plot Histograms for Bispectrum Coefficients
We plot histograms to compare the bispectrum coefficients of the 4-atom systems with CrFeCoNiCu. For CrCoNiCu (missing Fe), CrFeNiCu (missing Co), and CrFeCoCu (missing Ni), the bispectrum coefficients have similar distributions to those of CrFeCoNiCu. However, for FeCoNiCu (missing Cr) and CrFeCoNi (missing Cu), the bispectrum coefficient distributions differ from those of CrFeCoNiCu in range and peak location. 

We show plots for the first five bispectrum coefficients, but the code can be adjusted to show more or less coefficients.

In [ ]:
# iterate through all systems
for system in systems:
    # skip for CrFeCoNiCu
    if (system == 'CrFeCoNiCu'):
        continue
    # set number of bins
    num_bins = 50
    if (system == 'CrFeCoNi'):
        num_bins = 125
    # iterate through for first 5 bispectrum coefficients
    # to view more/less bispectrum coefficients, change indices of for loop
    for i in range(0, 5):
        #make histogram
        fig = go.Figure()
        fig.add_trace(go.Histogram(x=properties_dictionaries[system]['inputs'][:,i], name = system, nbinsx = num_bins))
        fig.add_trace(go.Histogram(x=properties_dictionaries['CrFeCoNiCu']['inputs'][:,i], name = 'CrFeCoNiCu', nbinsx = 50))
        fig.update_layout(barmode='overlay')
        fig.update_traces(opacity=0.75)
        fig.update_layout(
            xaxis_title="Bispectrum Coefficient - {}".format(i),
            yaxis_title="Frequency",
            font=dict(
                family="Times New Roman, monospace",
                size=24,
                color= "black"
            )
          )
        fig.show()

## 3. Plot Histograms for Output Properties

In the following cells, we plot histograms to show the distribution of output properties for the 4 atom systems compared to distributions for CrFeCoNiCu.

### Relaxed Vacancy Formation Energy

In [ ]:
# iterate through all systems
for system in systems:
    # skip for CrFeCoNiCu
    if (system == 'CrFeCoNiCu'):
        continue
    # set number of bins
    num_bins = 50
    if (system == 'CrFeCoNi'):
        num_bins = 75
    if (system == 'CrCoNiCu'):
        num_bins = 25
    #make histogram
    fig = go.Figure()
    fig.add_trace(go.Histogram(x=properties_dictionaries[system]['relaxed_vfe'], name = system, nbinsx = num_bins))
    fig.add_trace(go.Histogram(x=properties_dictionaries['CrFeCoNiCu']['relaxed_vfe'], name = 'CrFeCoNiCu', nbinsx = 50))
    fig.update_layout(barmode='overlay')
    fig.update_traces(opacity=0.75)
    fig.update_layout(
        xaxis_title="Relaxed VFE (eV/atom)",
        yaxis_title="Frequency",
        font=dict(
            family="Times New Roman, monospace",
            size=24,
            color= "black"
        )
      )
    fig.show()

### Cohesive Energy

In [ ]:
# iterate through all systems
for system in systems:
    # skip CoCrCuFeNi
    if (system == 'CrFeCoNiCu'):
        continue
    # set number of bins
    num_bins = 50
    if (system == 'CrFeCoNi'):
        num_bins = 75
    #make histogram 
    fig = go.Figure()
    fig.add_trace(go.Histogram(x=properties_dictionaries[system]['cohesive_energy'], name = system, nbinsx = num_bins))
    fig.add_trace(go.Histogram(x=properties_dictionaries['CrFeCoNiCu']['cohesive_energy'], name = 'CrFeCoNiCu', nbinsx = 50))
    fig.update_layout(barmode='overlay')
    fig.update_traces(opacity=0.75)
    fig.update_layout(
        xaxis_title="Cohesive Energy (eV/atom)",
        yaxis_title="Frequency",
        font=dict(
            family="Times New Roman, monospace",
            size=24,
            color= "black"
        )
      )
    fig.show()

### Pressure

In [ ]:
# iterate through all systems
for system in systems:
    # skip CoCrCuFeNi
    if (system == 'CrFeCoNiCu'):
        continue
    # set bin number
    num_bins = 50
    if (system == 'CrFeCoNi'):
        num_bins = 75
    if (system == 'CrCoNiCu'):
        num_bins = 25
    #make histogram 
    fig = go.Figure()
    fig.add_trace(go.Histogram(x=properties_dictionaries[system]['pressure'], name = system, nbinsx = num_bins))
    fig.add_trace(go.Histogram(x=properties_dictionaries['CrFeCoNiCu']['pressure'], name = 'CrFeCoNiCu', nbinsx = 50))
    fig.update_layout(barmode='overlay')
    fig.update_traces(opacity=0.75)
    fig.update_layout(
        xaxis_title="Pressure (GPa)",
        yaxis_title="Frequency",
        font=dict(
            family="Times New Roman, monospace",
            size=24,
            color= "black"
        )
      )
    fig.show()

### Volume

In [ ]:
# iterate through all systems
for system in systems:
    # skip CrFeCoNiCu
    if (system == 'CrFeCoNiCu'):
        continue
    # set number of bins
    num_bins = 50
    #make histogram 
    fig = go.Figure()
    fig.add_trace(go.Histogram(x=properties_dictionaries[system]['volume'], name = system, nbinsx = num_bins))
    fig.add_trace(go.Histogram(x=properties_dictionaries['CrFeCoNiCu']['volume'], name = 'CrFeCoNiCu', nbinsx = 50))
    fig.update_layout(barmode='overlay')
    fig.update_traces(opacity=0.75)
    fig.update_layout(
        xaxis_title="Volume (\u212B\u00b3)",
        yaxis_title="Frequency",
        font=dict(
            family="Times New Roman, monospace",
            size=24,
            color= "black"
        )
      )
    fig.show()